(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Recommender System for Amazon Electronics

In this assignment, we will be working with the [Amazon dataset](http://cs-people.bu.edu/kzhao/teaching/amazon_reviews_Electronics.tar.gz). You will build a recommender system to make predictions related to reviews of Electronics products on Amazon.

Your grades will be determined by your performance on the predictive tasks as well as a brief written report about the approaches you took.

This assignment should be completed **individually**.

## Files

**train.json** 1,000,000 reviews to be used for training. It is not necessary to use all reviews for training if doing so proves too computationally intensive. The fields in this file are:

* **reviewerID** The ID of the reviewer. This is a hashed user identifier from Amazon.

* **asin** The ID of the item. This is a hashed product identifier from Amazon.

* **overall** The rating of reviewer gave the item.

* **helpful** The helpfulness votes for the review. This has 2 subfields, 'nHelpful' and 'outOf'. The latter is the total number of votes this review received. The former is the number of those that considered the review to be helpful.

* **reviewText** The text of the review.

* **summary** The summary of the review.

* **unixReviewTime** The time of the review in seconds since 1970.

**meta.json** Contains metadata of the items:

* **asin** The ID of the item.

* **categories** The category labels of the item being reviewed.

* **price** The price of the item.

* **brand** The brand of the item.

**pairs_Rating.txt** The pairs (reviewerID and asin) on which you are to predict ratings.

**pairs_Purchase.txt** The pairs on which you are to predict whether a user purchased an item or not.

**pairs_Helpful.txt** The pairs on which you are to predict helpfulness votes. A third column in this file is the total number of votes from which you should predict how many were helpful.

**helpful.json** The review data associated with the helpfulness prediction test set. The 'nHelpful' field has been removed from this data since that is the value you need to predict above. This data will only be of use for the helpfulness prediction task.

**baseline.py** A simple baseline for each task.

## Tasks

**Rating prediction** Predict people's star ratings as accurately as possible for those (reviewerID, asin) pairs in 'pairs_Rating.txt'. Accuracy will be measured in terms of the [root mean-squared error (RMSE)](http://www.kaggle.com/wiki/RootMeanSquaredError).

**Purchase prediction** Predict given a (reviewerID, asin) pair from 'pairs_Purchase.txt' whether the user purchased the item (really, whether it was one of the items they reviewed). Accuracy will be measured in terms of the [categorization accuracy](http://www.kaggle.com/wiki/HammingLoss) (1 minus the Hamming loss).

**Helpfulness prediction** Predic whether a user's review of an item will be considered helpful. The file 'pairs_Helpful.txt' contains (reviewerID, asin) pairs with a third column containing the number of votes the user's review of the item received. You must predict how many of them were helpful. Accuracy will be measured in terms of the total [absolute error](http://www.kaggle.com/wiki/AbsoluteError), i.e. you are penalized one according to the difference |nHelpful - prediction|, where 'nHelpful' is the number of helpful votes the review actually received, and 'prediction' is your prediction of this quantity.

We set up competitions on Kaggle to keep track of your results compared to those of other members of the class. The leaderboard will show your results on half of the test data, but your ultimate score will depend on your predictions across the whole dataset.
* Kaggle competition: [rating prediction](https://inclass.kaggle.com/c/cs591-hw3-rating-prediction3) click here to [join](https://kaggle.com/join/datascience16rating)
* Kaggle competition: [purchase prediction](https://inclass.kaggle.com/c/cs591-hw3-purchase-prediction) click here to [join](https://kaggle.com/join/datascience16purchase)
* Kaggle competition: [helpfulness prediction](https://inclass.kaggle.com/c/cs591-hw3-helpful-prediction) click here to [join](https://kaggle.com/join/datascience16helpful)

## Grading and Evaluation

You will be graded on the following aspects.

* Your written report. This should describe the approaches you took to each of the 3 tasks. To obtain good performance, you should not need to invent new approaches (though you are more than welcome to) but rather you will be graded based on your decision to apply reasonable approaches to each of the given tasks. (**10pts** for each task)

* Your ability to obtain a solution which outperforms the baselines on the unseen portion of the test data. Obtaining full marks requires a solution which is substantially better (at least several percent) than baseline performance. (**10pts** for each task)

* Your ranking for each of the three tasks compared to other students in the class. (**5pts** for each task)

* Obtain a solution which outperforms the baselines on the seen portion of the test data (the leaderboard). 
(**5pts** for each task)

## Baselines

Simple baselines have been provided for each of the 3 tasks. These are included in 'baselines.py' among the files above. These 3 baselines operate as follows:

**Rating prediction** Returns the global average rating, or the user's average if you have seen them before in the training data.

**Purchase prediction** Finds the most popular products that account for 50% of purchases in the training data. Return '1' whenever such a product is seen at test time, '0' otherwise.

** Helpfulness prediction** Multiplies the number of votes by the global average helpfulness rate, or the user's rate if we saw this user in the training data.

Running 'baseline.py' produces 3 files containing predicted outputs. Your submission files should have the same format.

## Dataset Citation

**Image-based recommendations on styles and substitutes** J. McAuley, C. Targett, J. Shi, A. van den Hengel *SIGIR*, 2015

**Inferring networks of substitutable and complementary products** J. McAuley, R. Pandey, J. Leskovec *Knowledge Discovery and Data Mining*, 2015

-----------------

Solutions for all parts below. Labels in header comments

In [2]:
'''
    define all our variables.
'''
reviewerIdDict = {} # dictionary of reviewerId : other info from train_data
allAsinRatings = {} # dictionary of asinId : all ratings for that asin
averageAsinRatings = {} # dictionary of asinId : average rating for that product
allReviewerRatings = {} # dictionary of reviewerId : all ratings that reviewer has given
averageReviewerRating = {} # dictionary of reviewerId : average rating for all his products
globalRatingAverage = 0 # the average rating of all products in train.json
train_data = [] # a list of all lines in train.json
averagePurchases = 0 # the average purchases for a reviewer
meta_data = [] # a list of all line in meta.json
firstLine = True # represents if this is the first line in the file, to just write it back without doing any predicting
popularity = [] # a list of the sorted most bought items (aka most reviewed)
helpfulness = {} # dictionary of numVotes : average helpfulNumVotes for total votes = numVotes

In [3]:
'''
    parse json file into list
    train.json should be in the local directory
'''
import json
train_json = open('train.json', 'r')
train_data = []
for line in train_json:
    train_data.append(json.loads(line, strict=False))
train_json.close()

In [4]:
'''
    parse train data list to extract relevant information into dictionaries
'''
reviewerIdDict = {}    
allAsinRatings = {}
averageAsinRatings = {}
allReviewerRatings = {}
averageReviewerRatings = {}
globalRatingAverage = 0
averagePurchases = 0
helpfulness = {}
helpfulness[-1] = [] # used to represent global average number of helpful votes
for line in train_data:
    reviewerId = line[u'reviewerID']
    asinId = line['asin']
    overall = line['overall']
    globalRatingAverage = globalRatingAverage + overall
    helpful = line["helpful"]
    entryDict = {
    "asin" : asinId,
    "overall" : overall,
    "helpful" : {
        "Helpful" : helpful["nHelpful"],
        "outOf" : helpful["outOf"],
    },
    "reviewText" : line["reviewText"],
    "summary" : line["summary"],
    "unixReviewTime" : line["unixReviewTime"]
}
    if reviewerIdDict.get(reviewerId) is not None:
        allReviewerRatings[reviewerId].append(overall)
        reviewerIdDict[reviewerId].append(entryDict)
    else:
        allReviewerRatings[reviewerId] = [overall]
        reviewerIdDict[reviewerId] = [entryDict]
    
    if allAsinRatings.get(asinId) is not None:
        allAsinRatings[asinId].append(overall)
    else:
        allAsinRatings[asinId] = [overall]
    
    if helpfulness.get(helpful['outOf']) is None:
        helpfulness[helpful['outOf']] = [helpful['nHelpful']]
    else:
        helpfulness[helpful['outOf']].append(helpful['nHelpful'])
    helpfulness[-1].append(helpful['nHelpful'])

globalRatingAverage = globalRatingAverage / len(train_data)

keys = reviewerIdDict.keys()
for key in keys:
    averagePurchases = averagePurchases + len(reviewerIdDict[key])
averagePurchases = int(averagePurchases*1.0 / len(keys) + 0.5)

keys = allAsinRatings.keys()
for key in keys:
    averageAsinRatings[key] = float(sum(allAsinRatings[key])) / len(allAsinRatings[key])
    
keys = allReviewerRatings.keys()
for key in keys:
    averageReviewerRatings[key] = float(sum(allReviewerRatings[key])) / len(allReviewerRatings[key])
    
keys = helpfulness.keys()
for  key in keys:
    helpfulness[key] = float(sum(helpfulness[key])) / len(helpfulness[key])

In [5]:
'''
    parses the meta info into a dictionary
    meta.json should be in the local directory
        NOTE ON META.JSON!!! I slight edited meta.json to remove all ' symbols from categories.
        Using a different meta.json will break this method.
'''
meta_json = open('meta.json', 'r')
meta_data = []
for line in meta_json:
    meta_data.append(json.loads(line, strict=False))
meta_json.close()
item_info = {}
for line in meta_data:
    item_info[line['asin']] = [line['categories'], line['price']]

In [6]:
'''
    Calculates the predicted rating and writes it all to ratingPredictions.txt
    Assumes that pairs_Rating.txt is in the local directory
'''
ratingPredict = open('pairs_Rating.txt', 'r')
predictions = open('ratingPredictions.txt', 'w')
firstLine = True
for line in ratingPredict:
    if firstLine == True:
        # catch the header and just write it to the file
        predictions.write(line)
        firstLine = False
        continue
    # remove newline, will manually add later
    line = line.strip()
    (rid, aid) = line.split('-')
    prediction = 0
    if rid in reviewerIdDict and aid in allAsinRatings:
        ''' 
            assign weights to each average
            if reviewer is active, his global average weighs more
            if reviewer is not active, his global average weighs less
            asin average rating weighs inverse of reviewer
        '''
        reviewerBias = float(len(reviewerIdDict[rid])) / (len(reviewerIdDict[rid]) + len(allAsinRatings[aid]))
        asinBias = float(len(allAsinRatings[aid])) / (len(reviewerIdDict[rid]) + len(allAsinRatings[aid]))
        prediction = averageReviewerRatings[rid] * reviewerBias 
        prediction = prediction + averageAsinRatings[aid] * asinBias
        
        # weigh in the global average to increase accuracy slightly
        prediction = prediction*0.75 + 0.25*globalRatingAverage
    elif rid in reviewerIdDict and len(reviewerIdDict[rid]) > 10:
        '''
            if reviewer has more than 10 reviews, include his average.
            no asin available.
        '''
        prediction = averageReviewerRatings[rid]*0.8+ 0.2 * globalRatingAverage
    elif aid in allAsinRatings and len(allAsinRatings[aid]) > 15:
        '''
            if product has more than 15 reviews, include its average
            no reviewer available.
        '''
        prediction = averageAsinRatings[aid]*0.8 + 0.2 * globalRatingAverage
    else:
        # nothing significant to work with, use global average
        prediction = globalRatingAverage
    
    # precautionary last minute check to ensure prediction in range 1-5 and minimize error
    if prediction > 5:
        prediction = 5
    elif prediction < 1:
        prediction = 1
    # write to file
    predictions.write(line)
    predictions.write(',')
    predictions.write(str(prediction))
    predictions.write('\n')
ratingPredict.close()
predictions.close()

In [7]:
'''
    Makes dictionaries of the most popular elements for easy checking in purchase predictions.
'''
popularity = []

def getMostPopularProducts():
    keys = allAsinRatings.keys()
    for key in keys:
        popularity.append( (len(allAsinRatings[key]), key))
getMostPopularProducts()
popularity.sort(reverse=True)

top70Percent = {}
count = 0
while True:
    top70Percent[popularity[count][1]] = 1
    count = count + 1
    if count >= 0.7*len(popularity):
        break
        
top80Percent = {}
count = 0
while True:
    top80Percent[popularity[count][1]] = 1
    count = count + 1
    if count >= 0.8*len(popularity):
        break

In [8]:
'''
    Calculates the predicted purchase value and writes it all to purchasePredictions.txt
    Assumes that pairs_Purchase.txt is in the local directory
'''
pairsPurchase = open('pairs_Purchase.txt', 'r')
predictions = open('purchasePredictions.txt' ,'w')
firstLine = True
for line in pairsPurchase:
    if firstLine == True:
        # catch the header and just write it to the file
        predictions.write(line)
        firstLine = False
        continue
    else:
        prediction = 0
        line = line.strip('\n')
        (rid, asin) = line.split('-')
        reviewerInfo = reviewerIdDict.get(rid)
        if asin in top70Percent: 
            # include anything in the top 70 percent of all popular products
            prediction = 1
        elif asin in top80Percent and averageAsinRatings.get(asin) > 4.5:
            # include anything in the top 80 percent with a rating above 4.5 out of 5
            prediction = 1
        elif asin in top80Percent and rid in reviewerIdDict and len(reviewerIdDict[rid]) > 10:
            # include anything in the top 80 percent where the reviewer is known to buy many items
            prediction = 1
        else:
            prediction = 0
    predictions.write(line)
    predictions.write(',')
    predictions.write(str(prediction))
    predictions.write('\n')
pairsPurchase.close()
predictions.close()

In [9]:
'''
    Calculates the predicted number of helpful votes and writes it all to purchasePredictions.txt
    Assumes that pairs_Helpful.txt is in the local directory
'''
pairsHelpful = open('pairs_Helpful.txt', 'r')
predictions = open('helpfulnessPredictions.txt' ,'w')
firstLine = True
for line in pairsHelpful:
    if firstLine == True:
        # catch the header and just write it to the file
        predictions.write(line)
        firstLine = False
        continue
    else:
        line = line.strip()
        (rid, aid, outOf) = line.split('-')
        outOf = int(outOf)
        lower = outOf
        higher = outOf
        
        # grab the closest helpfulness score to outOf
        while helpfulness.get(lower) is None:
            lower = lower - 1
        while helpfulness.get(higher) is None:
            higher = higher + 1    
        if lower == outOf:
            prediction = helpfulness[outOf]
        elif outOf - lower < higher - outOf:
            prediction = helpfulness[lower]
        else:
            prediction = helpfulness[higher]
            
        # write
        predictions.write(line)
        predictions.write(',')
        predictions.write(str(prediction))
        predictions.write('\n')
pairsHelpful.close()
predictions.close()

In [9]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../custom.css", "r").read()
    return HTML(styles)
css_styling()